In [53]:
import pandas as pd
import dcube as dc
import numpy as np
import math
from scipy.stats import norm
token = ''
pro = dc.pro_api(token)
def read(file):
    df = pd.read_csv(file,index_col=0)
    df.index = [pd.to_datetime(str(i)) for i in df.index]
    df = df[df.index>pd.to_datetime('20180101')]
    return df
ptm_df=read(r"C:\Users\46997\Desktop\cache_data_ccb\CCB__ptm.csv")*250
ptm_df=ptm_df.round()
shibor = pro.query('Shibor', start_date='20180101', fields='date,1y').set_index('date').sort_index()
shibor = shibor/25000
shibor.index = pd.to_datetime(shibor.index)
r = shibor
s_df = read(r"C:\Users\46997\Desktop\cache_data_ccb\CCB__S__EODP__S_DQ_CLOSE.csv")
k_df = read(r"C:\Users\46997\Desktop\cache_data_ccb\CCB__conv_price.csv")
sigma_df = read(r"C:\Users\46997\Desktop\cache_data_ccb\CCB__S__STD240D.csv")
def intersect(indexs:list):
    index = indexs[0]
    for i in indexs[1:]:
        index = index.intersection(i)
    return index

def calculate_option_price_BS(S, K, T, r, sigma, option_type='call'):
    
    
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return option_price

In [3]:
pro.query('Shibor', start_date='20180101', fields='date,1y').set_index('date').sort_index()


NameError: name 'pro' is not defined

In [63]:
from tqdm import tqdm
def binomial(ptm,s,k,sigma,r):
    e = np.e
    ptm = int(ptm)
    n = ptm+1  #深度为到期时间+1
    df = pd.DataFrame(index=range(n),columns=range(n))
    u = e**sigma
    d = 1/u
    p = (e**r-d)/(u-d)
    print(s,k,u,d,p)
    df.iloc[0,0] = s
    #构建股价变动二叉树
    df = pd.DataFrame([[(d**i)*(u/d)**j for j in range(i+1)]+[np.nan]*(n-1-i) for i in range(n)])
    for i in range(n):
        df.iloc[n-1,i] = max(0,df.iloc[n-1,i]-k)  #计算
    for i in tqdm(range(n-2,-1,-1)):
        for j in range(i+1):
            p1 = (p*df.iloc[i+1,j+1]+(1-p)*df.iloc[i+1,j])*e**(-r)   #第i期持有期权价值
            p2 = max(0,df.iloc[i,j]-k)   #行使期权价值
            df.iloc[i,j] = max(p1,p2)
    df.to_csv(r"C:\Users\46997\Desktop\df.csv")
    print(df.iloc[0,0],calculate_option_price_BS(s,k,ptm,r,sigma))        
    return df.iloc[0,0]  

def binomial_factor(ptm_df,s_df,k_df,sigma_df,r_df):
    binomial_price = {}
    for day in s_df.index:
        r = r_df.loc[day,'1y']
        ptm = ptm_df.loc[day]
        s = s_df.loc[day]
        k = k_df.loc[day]
        sigma = sigma_df.loc[day]
        df = pd.concat([ptm,s,k,sigma],join='inner',axis=1).dropna().T
        value = df.apply(lambda col:binomial(col.iloc[0],col.iloc[1],col.iloc[2],col.iloc[3],r),axis=0)
        binomial[date] = value
    binomial_factor = pd.DataFrame(value).T
    return binomial_factor
        

In [62]:
binomial_factor(ptm_df,s_df,k_df,sigma_df,shibor)

5.77 7.24 1.0192435035130807 0.9811198173481086 0.5002156026930282


100%|████████████████████████████████████████████████████████████████████████████████| 495/495 [00:37<00:00, 13.18it/s]


8.849073658193727e-07 0.6772924383108738
21.65 42.8 1.0176983311673573 0.9826094525014536 0.5010256288306384


 26%|█████████████████████                                                           | 227/861 [00:46<02:10,  4.85it/s]


KeyboardInterrupt: 

In [68]:
a=pd.DataFrame([[1,2],[3,4]])
a.apply(lambda x:print(x,type(x)))

0    1
1    3
Name: 0, dtype: int64 <class 'pandas.core.series.Series'>
0    2
1    4
Name: 1, dtype: int64 <class 'pandas.core.series.Series'>


0    None
1    None
dtype: object

In [12]:
d=0.5
u=2
pd.DataFrame([[(d**i)*(u/d)**j for j in range(i+1)]+[np.nan]*(7-i) for i in range(8)])

,0,1,2,3,4,5,6,7
0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.500000,2.00000,NaN,NaN,NaN,NaN,NaN,NaN
2,0.250000,1.00000,4.000,NaN,NaN,NaN,NaN,NaN
3,0.125000,0.50000,2.000,8.0,NaN,NaN,NaN,NaN
4,0.062500,0.25000,1.000,4.0,16.0,NaN,NaN,NaN
5,0.031250,0.12500,0.500,2.0,8.0,32.0,NaN,NaN
6,0.015625,0.06250,0.250,1.0,4.0,16.0,64.0,NaN
7,0.007812,0.03125,0.125,0.5,2.0,8.0,32.0,128.0
